##UPDATES

***09/04/2021*** : Reorganized code.

***10/04/2021*** : Added callbacks in the model.fit function.

##Purpose

  In this notebook we create & train a N.N. model with **transfer learning based on DenseNet169**.

  The purpose of of the model is to classify plant's leaf images and extract decisions regarding their health.

  It is able to classify **38 different classes** ,13 different plants and some diseases that harm them .

  The Model is on it's own very light , but we are still going to convert it into a TF Lite version  in order to **run inference on a portable device , a RaspBerry Pi 4B** .

###Libraries and modules imports

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
from google.colab import files
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import DenseNet169 #57 MB acc Top1 0.762 top5 0.932
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D , Dropout

###Import a kaggle dataset directly to google colab (be wise with your time !).

In [4]:
####### SCRIPT TO DOWNLOAD KAGGLE DATASET IN GOOGLE COLAB ######### .
from google.colab import files
#Upload your kaggle.json file
files.upload()

!mkdir -p  ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#Copy the API command from kaggle .
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

#Unzip dataset ,always with -q (quiet)
!unzip -q /content/new-plant-diseases-dataset.zip

KeyboardInterrupt: 

###Auxiliar Functions

In [ ]:
#Necessary data preparation.
def data_prep():
    #Apply data augmentation on-the-fly on the training set to increase generalizability of the model.
    train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255,
                                                                shear_range = 0.2,
                                                                zoom_range = 0.2,
                                                                width_shift_range = 0.2,
                                                                height_shift_range = 0.2,
                                                                fill_mode="nearest")
    #Create the pipeline
    train_data = train_datagen.flow_from_directory(os.path.join(PATH,"train"),
                                                  target_size=(image_size,image_size),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode="categorical")
    # Normalize validation dataset's image rgb values.
    valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)
    #Create the pipeline
    valid_data = valid_datagen.flow_from_directory(os.path.join(PATH,"valid"),
                                                  target_size=(image_size,image_size),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode="categorical")
    return[train_data,valid_data]

In [ ]:
#Plot training learning curves for both train and validation.
def plot_training_curves(history):
        #Defining the metrics we will plot.
        train_acc=history.history['accuracy']
        val_acc=history.history['val_accuracy']
        train_loss = history.history['loss']
        val_loss = history.history['val_loss']

        #Range for the X axis.
        epochs = range(len(train_loss))

        fig,axis=plt.subplots(1,2,figsize=(20,8))#1 row, 2 col , width=20,height=8 inches.

        #Plotting Loss figures.
        plt.rcParams.update({'font.size': 22}) #configuring font size.
        plt.subplot(1,2,1) #plot 1st curve.
        plt.plot(epochs,train_loss,c="red",label="Training Loss") #plotting
        plt.plot(epochs,val_loss,c="blue",label="Validation Loss")
        plt.xlabel("Epochs") #title for x axis
        plt.ylabel("Loss")   #title for y axis
        plt.legend()

        #Plotting Accuracy figures.
        plt.subplot(1,2,2) #plot 2nd curve.
        plt.plot(epochs,train_acc,c="red",label="Training Acc") #plotting
        plt.plot(epochs,val_acc,c="blue",label="Validation Acc")
        plt.xlabel("Epochs")   #title for x axis
        plt.ylabel("Accuracy") #title for y axis
        plt.legend()

In [ ]:
#Make predictions on manually created test dataset folder. (OPTIONAL, different to initial dataset test set)
def test_model(path):
    i=0
    #Inversing class dictionary [keys,values]->[values,keys]
    labels = (train_data.class_indices)
    labels = dict((v,k) for k,v in labels.items())

    for filename in os.listdir(path) :
        i+=1
        file_path= path +"/" + str(pathlib.Path(filename))

        image = tf.keras.preprocessing.image.load_img(
                file_path,grayscale=False, color_mode="rgb",
                target_size=(image_size,image_size), interpolation="nearest")
        input_arr = keras.preprocessing.image.img_to_array(image)
        input_arr=input_arr/255
        input_arr = np.array([input_arr])  # Convert single image to a batch.
        prediction = model.predict(input_arr)


        a=np.argmax(prediction)
        results = labels[a]


        print("\n")
      # print("Predicted class number :", np.argmax(prediction))
        print("Actual class :" ,pathlib.Path(filename))
        print("predicted class name->  MobileV2:",results )


    print("\n Tested: " ,i, "new unseen images")

In [ ]:
def assign_callbacks(weights_file,patience,lr_factor):
  return[
      #Only save the weights that correspond to the minimum validation loss.
      tf.keras.callbacks.ModelCheckpoint(filepath=weights_file,
                                         monitor='val_accuracy',
                                         mode='max',
                                         save_best_only=True ,
                                         save_weights_only=True,
                                         verbose=0),
      #If val_loss doesn't improve for a number of epochs set with 'patience'
      #variable, training will stop to avoid overfitting.
      tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                       mode='min',
                                       patience=patience,
                                       verbose=1),
      #Learning rate is reduced by 'lr_factor' if val_loss stagnates
      #for a number of epochs set with 'patience/2' variable.
      tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                           mode='min',
                                           factor=lr_factor,
                                           patience=patience//2,
                                           min_lr=1e-6,
                                           verbose=1)]



##Experimental parameters

In [ ]:
EPOCHS=100 #Number of epochs to train the model.
BATCH_SIZE =32
PATIENCE=EPOCHS//5
LR_FACTOR=0.2
WEIGHTS_FILE = "weights.h5" #File that stores updated weights
image_size =224 #needs to comply with CNN input constraints.
PATH="/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)" #Path to dataset
TEST_PATH='/content/test/test'

In [ ]:
#The training class names.
class_names=["Apple___Apple_scab","Apple___Black_rot","Apple___Cedar_apple_rust","Apple___healthy",
             "Blueberry___healthy","Cherry_(including_sour)__Powedery_mildew","Cherry_(including_sour)__healthy",
             "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot","Corn_(maize)___Common_rust_","Corn_(maize)___Northern_Leaf_Blight",
             "Corn_(maize)___healthy","Grape___Black_rot","Grape___Esca_(Black_Measles)","Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
             "Grape___healthy","Orange___Haunglongbing_(Citrus_greening)","Peach___Bacterial_spot","Peach___healthy",
             "Pepper,_bell___Bacterial_spot","Pepper,_bell___healthy","Potato___Early_blight","Potato___Late_blight",
             "Potato___healthy","Raspberry___healthy","Soybean___healthy","Squash___Powdery_mildew",
             "Strawberry___Leaf_scorch","Strawberry___Healthy","Tomato___Bacterial_spot","Tomato___Early_blight","Tomato___Late_blight",
             "Tomato___Leaf_Mold","Tomato___Septoria_leaf_spot","Tomato___Spider_mites Two-spotted_spider_mite",
             "Tomato___Target_Spot","Tomato_Yellow_Leaf_Curl_Virus","Tomato_mosaic_virus","Tomato___healthy"]

##Define the Architecture

In [ ]:
#Create the whole neural network, pass the base model as parameter.
def create_model(base_model):
      x = base_model.output
      #Add a global spatial average pooling layer.
      x = GlobalAveragePooling2D()(x)
      #Add a fully-connected layer.
      x = Dense(1024, activation='relu')(x)
      #Add a dropout layer to decrease overfitting
      x=Dropout(0.2)(x)
      #Add a logistic layer with number_of_neurons=number_of_training_classes.
      predictions = Dense(38, activation='softmax')(x)

      #This is the model we will train
      model = Model(inputs=base_model.input, outputs=predictions
                                      ,name="Plant_Disease_Detector")

      #Freeze layers of the base model, we will only train the extra layers.
      #This way features already learned by the base model aren't lost.
      for layer in base_model.layers:
          layer.trainable = False

      #Assign a base learning rate for the Adam optimizer.
      base_learning_rate = 0.001
      model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate)
                    ,loss='categorical_crossentropy' ,metrics='accuracy')

      return model

###Execution

In [ ]:
#Prepare data
train_data,valid_data=data_prep()
#Define callbacks for training
callbacks=assign_callbacks(WEIGHTS_FILE,PATIENCE,LR_FACTOR)

In [ ]:
#Create the base pre-trained model, without the classification layers.
base_model =DenseNet169(weights='imagenet',
                        include_top=False ,input_shape=(224,224,3))
#Create the complete new model.
model=create_model(base_model)


In [ ]:
#Information about the model's layers.
model.summary()

In [ ]:
#Train the model on the dataset.
history=model.fit(x=train_data , batch_size=BATCH_SIZE,
                 epochs=EPOCHS , verbose=2 ,
                 validation_data=valid_data,
                 steps_per_epoch=20,
                 validation_steps=10,
                 callbacks=callbacks)

###Plotting the training curves.

In [ ]:
#Plot the Learning Curves from model training.
plot_training_curves(history)

###Convert to Tensorflow Lite and Quantize model.
 Converting the model to it's tensorflow lite equivalent and applying quantization provides better efficiency for the live detection on the Raspberry Pi.

 **Inference will be faster and it will need less memmory space to store model**.

In [ ]:
#Load optimal weights computed during training.
model.load_weights(WEIGHTS_FILE)
#Convert the model to it's Tensorflow Lite equivalent
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT] ##Quantized
tflite_model = converter.convert()

# Save the model.
with open('model.tfliteQuant', 'wb') as f:
  f.write(tflite_model)


####Scripts to save or restore the whole model. (OPTIONAL)

In [ ]:
#Save the whole model for experimentations.
!mkdir -p saved_model
model.save('saved_model/DenseNet169')

# my_model directory
!ls saved_model
# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/DenseNet169
# Zip it before download (So that you can download the whole directory at once )
!zip  -r /content/DenseNet169_In_Zip.zip /content/saved_model
print("Zip Ready")
# Download it
files.download('/content/DenseNet169_In_Zip.zip')

In [ ]:
#Restore an uploaded model for experimentations
files.upload()

!unzip -q /content/saved_model_In_Zip.zip
new_model = tf.keras.models.load_model('/content/content/saved_model/MobileNetV2')
# Check its architecture
new_model.summary()

###Evaluate the model's performance.

In [ ]:
#Load optimal weights computed during training.
model.load_weights(WEIGHTS_FILE)
eval=model.evaluate(valid_data)
print(eval)

In [ ]:
#Get predictions on the test_dataset you define with TEST_PATH
test_model(TEST_PATH)

###Future Work

Try different optimizers ,batch-size, dropout-rates and base learning rate.